In [1]:
import os, sys, re, json, time, unittest
import itertools, collections
import re
import operator
import numpy as np
import nltk
import math
import pandas as pd
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
stop_words = set(stopwords.words("english"))
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

In [2]:
#import csv as pandas array
#text_train_deps = pd.read_csv("text_train_deps.csv", encoding='utf8')
text_train_deps = pd.read_csv("./data/fin_text_train_deps.csv", encoding='utf8', dtype=str)


text_train_deps.head(5)

train_sentences = []
#combines all columns into 1 string
n = len(text_train_deps)
for row in text_train_deps['description'] + ';' + text_train_deps['host_about'] + ';' + \
text_train_deps['house_rules'] + ';' + text_train_deps['neighborhood_overview'] + ';' + \
text_train_deps['notes'] + ';' + text_train_deps['summary']:
    train_sentences.append(str(row))
print len(train_sentences)
print len(text_train_deps)

119615
119615


In [3]:
for each in train_sentences[:2]:
    print each

San Francisco Station AT T Park Safeway;None;None;None;None;San Francisco Station AT T Park Safeway
a Private Bedroom in a classic Victorian one block from famous 24th Street in Noe Valley easy to access the J Church Municipal Railway and the BART system Best Location you will find in the City We are steps from 24th Street in the heart of Noe Valley When family is not staying we would like to offer up our home to guests traveling through San Francisco the most beautiful city in the world We suggest two people because we are planning to use rent out other spaces within our home however if you have a more than than four we can make arrangements to rent the entire top floor which is equivalent to a flat Access to the dining room kitchen and bathroom The downstairs is a live work space that is occupied by the owner I d love to hear about your journeys and may be available during your stay I can assist with suggestions local and otherwise to enhance your journey Geographically Noe Valley re

In [ ]:
#creating entity features

corpus = train_sentences

def extract_entity_names(t):
    entity_names = []
    if hasattr(t, 'label') and t.label:
        # print t.label()
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

#getting tf and word count
document_entity_features = []

for row in corpus:
    sentences = nltk.sent_tokenize(row.decode('ascii', 'ignore'))
    # print '\nsentences:'
    # print sentences
    
    token_sent = [nltk.word_tokenize(sentence) for sentence in sentences]
    # print '\ntoken_sent:'
    # print token_sent
    
    tagged = [nltk.pos_tag(sentence) for sentence in token_sent]
    # print '\ntagged:'
    # print tagged
    
    chunked = nltk.ne_chunk_sents(tagged, binary=True)
    # print '\nchunked:'
    # print list(chunked)
    
    for tree in chunked:
        document_entity_features.append(extract_entity_names(tree))
    # print '\ndocument_entity_features:'
    # print document_entity_features

In [ ]:
#accumulate the "feature appearance" in document counts and feature word counts
document_count = {}
common_feature_words = {}
for doc in document_entity_features:
    current_document_count = {}
    for word in doc:
        if word.lower() not in stop_words:
            #gets the most common feature words
            if word.lower() not in common_feature_words:
                common_feature_words[word.lower()] = 1
            else:
                common_feature_words[word.lower()] += 1
            
    #accumulates the appearance of word in documents
            if word.lower() not in current_document_count:
                current_document_count[word.lower()] = 1

    for key in current_document_count.keys():
        if key not in document_count:
            document_count[key] = current_document_count[key]
        elif key in document_count:
            document_count[key] += current_document_count[key]
            

In [ ]:
#pulls the least common 10000 words
#set the number of words
n = 10000

print len(common_feature_words)
keys = sorted(common_feature_words.items(), key=operator.itemgetter(1))
sorted_keys = [str(key[0]) for key in keys]
least_common_keys = sorted_keys[:n]
#print common_feature_words
least_common_entities = {}
for key in least_common_keys:
    least_common_entities[key] = common_feature_words[key]

print len(least_common_entities)
print least_common_entities


In [ ]:
sid = SentimentIntensityAnalyzer()
def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

#getting tf and word count
total_word_count = 0
tfidf_sentiment = []
#for pandas export purposes
tfidf = []
sentiment = []

for row in corpus:
    current_entities = []
    temp_tfidf = []
    sentences = nltk.sent_tokenize(row.decode('ascii', 'ignore'))
    token_sent = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged = [nltk.pos_tag(sentence) for sentence in token_sent]
    chunked = nltk.ne_chunk_sents(tagged, binary=True)
    
    for tree in chunked:
        current_entities.extend(extract_entity_names(tree))
    #gets chunked word count
    for word in sentence:
        total_word_count += 1   
        
    if len(current_entities) > 0:
        
        #counts the amount of times the word appears in the sentence
        current_word_count = {}
        for word in current_entities:
            lower_word = word.lower()
            if lower_word in least_common_entities:
                if lower_word not in current_word_count:
                    current_word_count[lower_word] = 1
                elif lower_word in current_word_count:
                    current_word_count[lower_word] += 1
        if len(current_word_count) > 0:    
            
            for word in current_word_count.keys():
                tf = float(current_word_count[word])/total_word_count
                idf = math.log(len(corpus)/(1+float(document_count[word])))
                tf_idf = tf*idf
                temp_tfidf.append([word,tf_idf])
        else:
                temp_tfidf.append(["no_keys", str(0.0)])
    else:
        temp_tfidf.append(["no_keys", str(0.0)])
        
    ss = sid.polarity_scores(row)
    tfidf_sentiment.append([temp_tfidf, ss])    
    
    #for exporting pandas purposes
    tfidf.append(temp_tfidf)
    sentiment.append(ss)

In [ ]:
print len(tfidf_sentiment)
#for each in tfidf_sentiment:
#    print each

In [ ]:
for each in tfidf_sentiment[:1000]:
    print each

In [ ]:
from pandas import DataFrame

pd_tfidf_sentiment = DataFrame.from_records(tfidf_sentiment)
pd_tfidf_sentiment.columns = ["keys", "sentiment"]
pd_tfidf_sentiment.head(5)

pd_tfidf_sentiment.to_csv('pd_tfidf_sentiment.csv', encoding='utf-8')

In [ ]:
test = pd.read_csv("pd_tfidf_sentiment.csv")
test.head(5)
print len(text_train_deps)
print len(test)

In [ ]:
test